# AE3: Optymalizacja wag w sieci MLP z użyciem algorytmu genetycznego
Adrianna Grudzień

In [168]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

In [136]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def softmax(x):
    if x.shape[1] == 1:
        return np.exp(x) / np.sum(np.exp(x))
    if x.shape[1] > 1:
        return np.exp(x) / (np.sum(np.exp(x), axis=1).reshape(
            (x.shape[0], 1)) @ np.ones((1, x.shape[1])))
    
def mean_error(y_pred_df, hot_enc_classes):
    '''Błąd klasyfikacji'''
    return np.mean(np.mean(np.abs(y_pred_df-hot_enc_classes), axis=1), axis=0)

In [59]:
softmax(np.array([2,5,3]))

array([0.04201007, 0.84379473, 0.1141952 ])

In [144]:
class Network:
    def __init__(self, sizes):
        self.sizes = sizes
        self.num_layers = len(sizes)
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        
        self.mse = [] # mse


    # def feedforward0(self,x):
    #     x = np.matrix(x)
    #     for b, w in zip(self.biases, self.weights):
    #         x = sigmoid(np.dot(x,w.T)+b.T)
    #     return x
    
    def feedforward(self, x):
        x = np.matrix(x)
        for i in range(len(self.biases)-1):
            x = sigmoid(np.dot(x, self.weights[i].T)+self.biases[i].T)
        # dla ostatniej warstwy - softmax:
        i = len(self.biases)-1
        x = softmax(np.dot(x,self.weights[i].T)+self.biases[i].T)
        return x

## Zbiór `iris`

In [23]:
iris = pd.read_csv('data/iris.data', header=None)
iris.columns = ['sepal_length','sepal_widrh','petal_length','petal_width','class']
iris.head(3)

,sepal_length,sepal_widrh,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa


In [108]:
print(np.unique(iris['class']))
class_dict = {'Iris-setosa':0, 'Iris-versicolor':1, 'Iris-virginica':2}
iris_x = iris[['sepal_length','sepal_widrh','petal_length','petal_width']]
iris_y = iris['class'].map(class_dict) # mapowanie nazw klas w liczby

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


### One-hot encoding klas

In [131]:
hot_enc_classes = np.zeros(shape=(len(iris_y),3), dtype='int')
for i in range(len(iris_y)):
    hot_enc_classes[i][iris_y[i]] = 1
hot_enc_classes = pd.DataFrame(hot_enc_classes)
hot_enc_classes.head(3)

,0,1,2
0,1,0,0
1,1,0,0
2,1,0,0


3 klasy.

In [142]:
mlp_iris = Network([4,3,3])
y_pred = mlp_iris.feedforward(iris_x)

In [143]:
y_pred_df = pd.DataFrame(y_pred)
print(y_pred_df.head(3))

mean_error(y_pred_df,hot_enc_classes)

          0         1         2
0  0.565719  0.030387  0.403893
1  0.568183  0.030644  0.401173
2  0.565424  0.030970  0.403605


0.433655928396947

In [172]:
class AE:
    def __init__(self, N, architecture, verbose_step=10):
        self.N = N
        self.architecture = architecture # architektura sieci MLP
        self.verbose_step = verbose_step
        self.population = [] # lista sieci
        self._initialize_population()
        
        self.new_population = copy.deepcopy(self.population) # populacja, do której będą dodawane nowe osobniki

        
    def _initialize_population(self):
        for i in range(self.N):
            self.population.append(Network(self.architecture))
            
    def select_radomly_from_population(self):
        '''Losowanie dowolnego osobnika z populacji.'''
        randomNetwork = np.random.randint(self.N)
        return self.population[randomNetwork]
    
    def cross(self, prob_boundary=0.7):
        '''
        Krzyżowanie.
        - losujemy 2 sieci x1, x2 i tworzymy nowego osobnika o tej samej architekturze;
        wartości wag - z połowy wartości x1 (dla nieparzystej liczby warstw - sufit)
        oraz drugiej połowy wartości x2 (dla nieparzystej liczby warstw - podłoga)
        ''' 
        drawn_prob = np.random.uniform(0,1) # wylosowane prawdopodobieństwo krzyżowania dla danych osobników
        if drawn_prob <= prob_boundary:
            # losowanie osobników
            x1 = self.select_radomly_from_population()
            x2 = self.select_radomly_from_population()
            # określenie ile wartości z x1, x2 bierzemy do utworzenia nowego osobnika n2=len(self.architecture)-n1
            n1 = int(np.ceil(len(self.architecture)/2))

            # łączenie osobników
            x_new = Network(self.architecture)
            for i in range(len(self.architecture)):
                if i < n1:
                    x_new.weights[i] = x1.weights[i]
                    x_new.biases[i] = x1.biases[i]
                else:
                    x_new.weights[i] = x2.weights[i]
                    x_new.biases[i] = x2.biases[i]

            # dodanie nowego osobnika do populacji
            self.new_population.append(x_new)
            
    def mutate(self, x, prob_boundary=0.2):
        '''Mutacja gaussowska - wylosowanie zgodnie z rozkładem normalnym 
        przesunięcia wektora w każdym z kierunków
        
        x - mutowany osobnik
        prob_boundary - prawdopodobieństwo mutacji na pojedynczy osobniku (prawd. graniczne)
        '''
    
        drawn_prob = np.random.uniform(0,1) # wylosowane prawdopodobieństwo mutacji dla danego osobnika
        if drawn_prob <= prob_boundary:
            vector_displacement = np.random.normal(0, 1)
            x_new = np.array([x[i]+vector_displacement for i in range(self.dim)]).reshape((self.dim, 1))

            # dodanie nowego osobnika do populacji
            self.new_population = np.concatenate((self.new_population, x_new), axis=1)
    
    def fit(self, n_epochs=10):
        for i in range(n_epochs):
            # krzyżowanie:
            for j in range(self.N):
                self.cross()
            # mutacja:
            for j in range(self.N):
                x = self.population[j]
                self.mutate()
            # selekcja

In [173]:
ae = AE(N=50, architecture=[4,3,3])
ae.fit(n_epochs=10)